In [21]:
import pandas as pd

# Ruta del archivo CSV original
ruta_archivo_original = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_brutos/a_coruña.csv"

# Leer el archivo CSV original en un DataFrame
dataframe_original = pd.read_csv(ruta_archivo_original)

# Seleccionar las columnas necesarias
columnas_seleccionadas = ["Tipo de inmueble", "Descripción", "Dirección", "Precio", "Habitaciones"]
dataframe_nuevo = dataframe_original[columnas_seleccionadas]

# Ruta del nuevo archivo CSV
ruta_archivo_nuevo = "a_coruña_clean.csv"  # Reemplaza con la ruta donde deseas guardar el nuevo archivo

# Guardar el nuevo DataFrame en un nuevo archivo CSV
dataframe_nuevo.to_csv(ruta_archivo_nuevo, index=False)

print("Nuevo archivo CSV creado con éxito.")


Nuevo archivo CSV creado con éxito.


---
---

## codigo para limpiar csv y sacar m2 de descripcion

In [22]:
import os
import pandas as pd
import re
import random

# Ruta de la carpeta que contiene los archivos CSV brutos
carpeta_csv_brutos = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_brutos"

# Ruta de la carpeta donde se guardarán los archivos CSV limpios
carpeta_csv_limpio = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio"

# Lista de archivos CSV en la carpeta
archivos_csv = os.listdir(carpeta_csv_brutos)

# Función para extraer 'm2' o 'metros cuadrados' y los números decimales detrás
def extraer_m2(descripcion):
    match = re.search(r'(\d+(\.\d+)?)\s*(m2|metros\s*cuadrados?)', descripcion, re.I)
    if match:
        return match.group(1)
    return None

# Función para generar números aleatorios enteros en función del número de habitaciones
def numero_aleatorio_por_habitaciones(num_habitaciones):
    if num_habitaciones is None:
        return random.randint(0, 10)  # Valor aleatorio si no se conoce el número de habitaciones
    num_habitaciones = int(num_habitaciones)  # Convertir a entero
    if num_habitaciones == 0:
        return random.randint(20, 50)
    elif num_habitaciones == 1:
        return random.randint(40, 60)
    elif num_habitaciones == 2:
        return random.randint(60, 80)
    elif num_habitaciones == 3:
        return random.randint(80, 120)
    elif num_habitaciones == 4:
        return random.randint(120, 160)
    elif num_habitaciones == 5:
        return random.randint(160, 200)
    elif num_habitaciones == 6:
        return random.randint(200, 250)
    elif num_habitaciones == 7:
        return random.randint(250, 300)
    elif num_habitaciones == 8:
        return random.randint(300, 400)
    elif num_habitaciones >= 9:
        return random.randint(400, 700)  # Como referencia, pero puedes ajustarlo según tus necesidades
    else:
        return None

# Iterar sobre los archivos CSV en la carpeta
for archivo in archivos_csv:
    ruta_archivo_original = os.path.join(carpeta_csv_brutos, archivo)

    # Leer el archivo CSV original en un DataFrame
    dataframe_original = pd.read_csv(ruta_archivo_original, low_memory=False)

    # Verificar si la columna 'Descripción' existe en el DataFrame
    if 'Descripción' in dataframe_original.columns:
        # Limpiar la columna 'Descripción' rellenando los valores nulos con una cadena vacía
        dataframe_original['Descripción'].fillna('', inplace=True)

        # Aplicar la función para extraer 'm2' a la columna 'Descripción'
        dataframe_original['m2'] = dataframe_original['Descripción'].apply(extraer_m2)

    # Convertir la columna "Habitaciones" a valores numéricos enteros
    dataframe_original['Habitaciones'] = pd.to_numeric(dataframe_original['Habitaciones'], errors='coerce').astype('Int64')

    # Rellenar filas sin número con números aleatorios según el número de habitaciones
    dataframe_original['Habitaciones'].fillna(random.randint(0, 10), inplace=True)  # Rellenar habitaciones vacías con un valor aleatorio entre 0 y 10
    dataframe_original['m2'].fillna(dataframe_original['Habitaciones'].apply(numero_aleatorio_por_habitaciones), inplace=True)

    # Seleccionar las columnas necesarias
    columnas_seleccionadas = ["Tipo de inmueble", "Dirección", "Precio", "Habitaciones", "m2"]

    # Extraer el nombre de la provincia del nombre del archivo (sin la extensión .csv)
    nombre_provincia = os.path.splitext(archivo)[0]

    # Crear la ruta completa para el archivo CSV limpio
    ruta_archivo_nuevo = os.path.join(carpeta_csv_limpio, f"{nombre_provincia}_limpio.csv")

    # Guardar el DataFrame final en un nuevo archivo CSV
    dataframe_final = dataframe_original[columnas_seleccionadas]
    dataframe_final.to_csv(ruta_archivo_nuevo, index=False)

    print(f"Nuevo archivo CSV {nombre_provincia}_limpio.csv creado con éxito en {ruta_archivo_nuevo}.")


Nuevo archivo CSV huesca_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/huesca_limpio.csv.
Nuevo archivo CSV murcia_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/murcia_limpio.csv.
Nuevo archivo CSV lleida_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/lleida_limpio.csv.
Nuevo archivo CSV guadalajara_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/guadalajara_limpio.csv.
Nuevo archivo CSV barcelona_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/barcelona_limpio.csv.
Nuevo archivo CSV las_palmas_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/las_palmas_limpio.csv.
Nuevo archivo CSV palencia_limpio.csv creado con éxito en /workspaces/028-PROYECTO_FINAL_VIVIE

## añadir a csv_limpio columna provincia y c. autonoma


In [25]:
import os
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split

# Ruta de la carpeta donde se encuentran los archivos CSV limpios
ruta_csv_limpio = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio"

# Ruta de la carpeta donde se guardarán los archivos train y test seleccionados
ruta_guardar = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA"

# Lista de provincias
provincias = [
    "alava", "albacete", "alicante", "almeria", "asturias", "avila", "badajoz",
    "barcelona", "burgos", "caceres", "cadiz", "cantabria", "castellon",
    "ciudad_real", "cordoba", "cuenca", "girona", "granada", "guadalajara",
    "guipuzcoa", "huelva", "huesca", "islas_baleares", "jaen", "la_coruna",
    "la_rioja", "las_palmas", "leon", "lerida", "lugo", "madrid", "malaga",
    "murcia", "navarra", "ourense", "palencia", "pontevedra", "salamanca",
    "santa_cruz_de_tenerife", "segovia", "sevilla", "soria", "tarragona",
    "teruel", "toledo", "valencia", "valladolid", "vizcaya", "zamora", "zaragoza"
]

# Número de mejores características a seleccionar
num_mejores_caracteristicas = 3

for provincia in provincias:
    # Cargar el archivo CSV de la provincia
    archivo_csv = os.path.join(ruta_csv_limpio, f"{provincia}.csv")
    dataframe = pd.read_csv(archivo_csv)

    # Dividir el DataFrame en características (X) y variable objetivo (y)
    X = dataframe.drop(columns=["Precio"])
    y = dataframe["Precio"]

    # Aplicar SelectKBest para seleccionar las mejores características
    k_best_selector = SelectKBest(score_func=f_regression, k=num_mejores_caracteristicas)
    X_selected = k_best_selector.fit_transform(X, y)

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Crear nombres de archivo para guardar los conjuntos de entrenamiento y prueba
    archivo_train = os.path.join(ruta_guardar, f"{provincia}_train_selected.csv")
    archivo_test = os.path.join(ruta_guardar, f"{provincia}_test_selected.csv")

    # Guardar los conjuntos de entrenamiento y prueba en archivos CSV
    pd.DataFrame(X_train, columns=[f"feature_{i}" for i in range(num_mejores_caracteristicas)]).to_csv(archivo_train, index=False)
    pd.DataFrame(X_test, columns=[f"feature_{i}" for i in range(num_mejores_caracteristicas)]).to_csv(archivo_test, index=False)

    print(f"Archivos guardados para {provincia}: {archivo_train}, {archivo_test}")


## eliminamos tildes y acentuaciones


In [28]:
import os
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split

# Ruta de la carpeta donde se encuentran los archivos CSV limpios
ruta_csv_limpio = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio"

# Ruta de la carpeta donde se guardarán los archivos train y test seleccionados
ruta_guardar = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA"

# Lista de provincias
provincias = [
    "alava", "albacete", "alicante", "almeria", "asturias", "avila", "badajoz",
    "barcelona", "burgos", "caceres", "cadiz", "cantabria", "castellon",
    "ciudad_real", "cordoba", "cuenca", "girona", "granada", "guadalajara",
    "guipuzcoa", "huelva", "huesca", "islas_baleares", "jaen", "la_coruna",
    "la_rioja", "las_palmas", "leon", "lerida", "lugo", "madrid", "malaga",
    "murcia", "navarra", "ourense", "palencia", "pontevedra", "salamanca",
    "santa_cruz_de_tenerife", "segovia", "sevilla", "soria", "tarragona",
    "teruel", "toledo", "valencia", "valladolid", "vizcaya", "zamora", "zaragoza"
]

# Número de mejores características a seleccionar
num_mejores_caracteristicas = 3

for provincia in provincias:
    # Cargar el archivo CSV de la provincia
    archivo_csv = os.path.join(ruta_csv_limpio, f"{provincia}.csv")
    dataframe = pd.read_csv(archivo_csv)

    # Dividir el DataFrame en características (X) y variable objetivo (y)
    X = dataframe.drop(columns=["Precio"])
    y = dataframe["Precio"]

    # Aplicar SelectKBest para seleccionar las mejores características
    k_best_selector = SelectKBest(score_func=f_regression, k=num_mejores_caracteristicas)
    X_selected = k_best_selector.fit_transform(X, y)

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Crear nombres de archivo para guardar los conjuntos de entrenamiento y prueba
    archivo_train = os.path.join(ruta_guardar, f"{provincia}_train_selected.csv")
    archivo_test = os.path.join(ruta_guardar, f"{provincia}_test_selected.csv")

    # Guardar los conjuntos de entrenamiento y prueba en archivos CSV
    pd.DataFrame(X_train, columns=[f"feature_{i}" for i in range(num_mejores_caracteristicas)]).to_csv(archivo_train, index=False)
    pd.DataFrame(X_test, columns=[f"feature_{i}" for i in range(num_mejores_caracteristicas)]).to_csv(archivo_test, index=False)

    print(f"Archivos guardados para {provincia}: {archivo_train}, {archivo_test}")


FLOAT A ENTERO EN PRECIO

In [35]:
import os
import pandas as pd

# Ruta de la carpeta donde se encuentran los CSV limpios
ruta_carpeta_csv_limpio = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio"

# Iterar sobre los archivos en la carpeta y cambiar los valores de la columna 'Precio' a int64
for archivo in os.listdir(ruta_carpeta_csv_limpio):
    if archivo.endswith(".csv"):
        ruta_archivo = os.path.join(ruta_carpeta_csv_limpio, archivo)
        df = pd.read_csv(ruta_archivo)
        df['Precio'] = df['Precio'].astype('Int64')  # Convertir la columna 'Precio' a int64
        df.to_csv(ruta_archivo, index=False)  # Guardar el archivo CSV actualizado


---
---
---

## sacar train/test de csv_limpio

In [7]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Directorio de entrada (csv_limpio)
input_dir = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio"

# Directorio de salida
output_dir = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA"

# Listado de provincias
provincias = [
    "alava_limpio", "albacete_limpio", "alicante_limpio", "almeria_limpio", 
    "asturias_limpio", "avila_limpio", "badajoz_limpio", "barcelona_limpio", 
    "burgos_limpio", "caceres_limpio", "cadiz_limpio", "cantabria_limpio", 
    "castellon_limpio", "ciudad_real_limpio", "cordoba_limpio", "cuenca_limpio", 
    "girona_limpio", "granada_limpio", "guadalajara_limpio", "guipuzcoa_limpio", 
    "huelva_limpio", "huesca_limpio", "islas_baleares_limpio", "jaen_limpio", 
    "a_coruña_limpio", "la_rioja_limpio", "las_palmas_limpio", "leon_limpio", 
    "lleida_limpio", "lugo_limpio", "madrid_limpio", "malaga_limpio", 
    "murcia_limpio", "navarra_limpio", "ourense_limpio", "palencia_limpio", 
    "pontevedra_limpio", "salamanca_limpio", "santa_cruz_de_tenerife_limpio", 
    "segovia_limpio", "sevilla_limpio", "soria_limpio", "tarragona_limpio", 
    "teruel_limpio", "toledo_limpio", "valencia_limpio", "valladolid_limpio", 
    "vizcaya_limpio", "zamora_limpio", "zaragoza_limpio"
]

# Iterar sobre las provincias
for provincia in provincias:
    # Cargar el archivo CSV correspondiente
    csv_file = os.path.join(input_dir, f"{provincia}.csv")
    df = pd.read_csv(csv_file)
    
    # Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Guardar los conjuntos de entrenamiento y prueba
    train_file = os.path.join(output_dir, f"{provincia}_train_selected.csv")
    test_file = os.path.join(output_dir, f"{provincia}_test_selected.csv")
    
    train_df.to_csv(train_file, index=False)
    test_df.to_csv(test_file, index=False)
    
    print(f"Guardados conjuntos de entrenamiento y prueba para {provincia} en {train_file} y {test_file}")


Guardados conjuntos de entrenamiento y prueba para alava_limpio en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/alava_limpio_train_selected.csv y /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/alava_limpio_test_selected.csv
Guardados conjuntos de entrenamiento y prueba para albacete_limpio en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/albacete_limpio_train_selected.csv y /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/albacete_limpio_test_selected.csv
Guardados conjuntos de entrenamiento y prueba para alicante_limpio en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/alicante_limpio_train_selected.csv y /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/alicante_limpio_test_selected.csv
Guardados conjuntos de entrenamiento y prueba para almeria_limpio en /workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/BLABLA/almeria_limpio_train_selected.csv y /work

---
---
---

In [8]:
import pandas as pd

# Ruta del archivo CSV
csv_file_path = "/workspaces/028-PROYECTO_FINAL_VIVIENDAS-main/src/PROVINCIAS/csv_limpio/alava_limpio.csv"

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path)

# Calcular los valores mínimos y máximos del precio
min_price = df['Precio'].min()
max_price = df['Precio'].max()

# Imprimir los valores mínimos y máximos
print("Valor mínimo del precio:", min_price)
print("Valor máximo del precio:", max_price)


Valor mínimo del precio: 0
Valor máximo del precio: 945588227
